In [1]:
!pip install tensorflow


In [51]:
print("created by: \u0930\u094b\u0936\u0928\u0940 \u0917\u094c\u095c")

created by: रोशनी गौड़


In [31]:
import pandas as pd
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.utils import to_categorical, pad_sequences
import numpy as np

import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda
from sklearn.metrics.pairwise import euclidean_distances


In [33]:
data = [
    "Natural Language Processing is a field of Artificial Intelligence.",
    "Word embeddings help computers understand human language.",
    "The CBOW model is a part of Word2Vec technique.",
    "CBOW predicts the target word using surrounding context words.",
    "Skip Gram is another architecture of Word2Vec.",
    "Word2Vec is widely used in NLP applications.",
    "Embedding layers in deep learning are used to represent words.",
    "CBOW is faster and works better with frequent words."
]
#A

In [35]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

tokenizer = text.Tokenizer()
tokenizer.fit_on_texts([[w for w in text.text_to_word_sequence(doc) if w not in stop_words] for doc in data])

word2id = tokenizer.word_index
word2id['PAD'] = 0 
id2word = {v: k for k, v in word2id.items()}

wids = [[word2id.get(w, word2id['PAD']) for w in text.text_to_word_sequence(doc) if w not in stop_words]for doc in data] #id to words

vocab_size = len(word2id)
embed_size = 100
window_size = 2 

print("Vocabulary Size:", vocab_size)
print("Sample Vocabulary:", list(word2id.items())[:10])

Vocabulary Size: 41
Sample Vocabulary: [('cbow', 1), ('word2vec', 2), ('words', 3), ('language', 4), ('word', 5), ('used', 6), ('natural', 7), ('processing', 8), ('field', 9), ('artificial', 10)]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Roshnee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size * 2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word = []            
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([words[i] 
                                  for i in range(start, end) 
                                  if 0 <= i < sentence_length and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            y = to_categorical(label_word, vocab_size) # 1 hot encoded centre word
            yield (x, y)
        #B

In [37]:
i = 0
for x, y in generate_context_word_pairs(wids, window_size, vocab_size):
    if 0 not in x[0]:  # skip padded ones
        print("Context (X):", [id2word[w] for w in x[0]], "-> Target (Y):", id2word[np.argmax(y[0])])
        i += 1
        if i == 5:
            break

Context (X): ['natural', 'language', 'field', 'artificial'] -> Target (Y): processing
Context (X): ['language', 'processing', 'artificial', 'intelligence'] -> Target (Y): field
Context (X): ['word', 'embeddings', 'computers', 'understand'] -> Target (Y): help
Context (X): ['embeddings', 'help', 'understand', 'human'] -> Target (Y): computers
Context (X): ['help', 'computers', 'human', 'language'] -> Target (Y): understand


In [38]:
cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_shape=(window_size*2,)))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation="softmax"))
cbow.compile(loss="categorical_crossentropy", optimizer="adam")
#C
print(cbow.summary())

C:\Users\Roshnee\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 4, 100)              │           4,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_1 (Lambda)                    │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 41)                  │           4,141 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,241 (32.19 KB)

 Trainable params: 8,241 (32.19 KB)

 Non-trainable params: 0 (0.00 B)

None


In [39]:
'''X, Y = [], []
for x, y in generate_context_word_pairs(wids, window_size, vocab_size):
    X.append(x[0])  # 1 d array
    Y.append(y[0])  

X = np.array(X)
Y = np.array(Y)

print("Training Data Shape:", X.shape, Y.shape)

history = cbow.fit(
    X, Y,
    epochs=10,
    batch_size=32, #(32+17)
    verbose=0
)'''

for epoch in range(1, 50):
    loss = 0.
    i = 0
    for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 100000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:', epoch, '\tLoss:', loss)
    print()

Epoch: 1 	Loss: 181.98066759109497

Epoch: 2 	Loss: 181.45550847053528

Epoch: 3 	Loss: 180.33871245384216

Epoch: 4 	Loss: 179.12486481666565

Epoch: 5 	Loss: 177.76869773864746

Epoch: 6 	Loss: 176.23424887657166

Epoch: 7 	Loss: 174.49693965911865

Epoch: 8 	Loss: 172.54545974731445

Epoch: 9 	Loss: 170.38276362419128

Epoch: 10 	Loss: 168.02441811561584

Epoch: 11 	Loss: 165.49482226371765

Epoch: 12 	Loss: 162.821932554245

Epoch: 13 	Loss: 160.03305459022522

Epoch: 14 	Loss: 157.15254831314087

Epoch: 15 	Loss: 154.2017364501953

Epoch: 16 	Loss: 151.199720621109

Epoch: 17 	Loss: 148.16437792778015

Epoch: 18 	Loss: 145.1125738620758

Epoch: 19 	Loss: 142.06028652191162

Epoch: 20 	Loss: 139.02212023735046

Epoch: 21 	Loss: 136.01116108894348

Epoch: 22 	Loss: 133.03862261772156

Epoch: 23 	Loss: 130.11375546455383

Epoch: 24 	Loss: 127.24388909339905

Epoch: 25 	Loss: 124.4348042011261

Epoch: 26 	Loss: 121.69065451622009

Epoch: 27 	Loss: 119.01434564590454

Epoch: 28 	Loss: 

In [40]:
weights = cbow.get_weights()[0]
weights = weights[1:] #1st row
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(40, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
word2vec,0.352018,-0.434074,-0.104973,0.173810,-0.383382,-0.055270,0.417691,0.222921,0.227587,0.309437,...,0.378679,0.728197,0.035561,0.441801,0.360132,-0.184294,-0.204133,-0.011996,0.606085,-0.333869
words,-0.284951,-0.309418,0.624363,-0.489778,0.084887,0.361340,0.233101,-0.544445,0.542537,0.731797,...,0.074275,0.490322,0.472407,0.066562,0.689298,-0.280289,-0.207738,0.344961,0.551165,-0.409315
language,-0.513743,-0.008611,0.108116,-0.017404,0.256702,0.065605,-0.117492,-0.654128,-0.597783,0.551359,...,0.016540,-0.713183,-0.283292,0.341617,0.216707,0.151271,0.522702,0.130717,-0.001808,-0.445739
word,0.169356,0.507206,0.210628,0.504095,0.108347,-0.281278,0.218709,0.207728,0.324459,0.191649,...,-0.197952,0.154228,0.475912,0.548594,0.124993,0.482804,0.092264,-0.621354,0.213806,-0.184896
used,-0.006296,-0.305421,0.142024,0.237366,-0.331905,0.587903,-0.533791,0.482473,0.168174,-0.133638,...,0.405431,0.514301,-0.366554,0.285204,-0.459063,0.692945,-0.090628,-0.177005,0.462135,0.433044


In [41]:
user_word = input("Enter a word to find similar words: ").strip().lower()

if user_word in word2id:
    # Calculate distance/similarity
    distance_matrix = euclidean_distances(weights)
    similar = [id2word[idx] for idx in distance_matrix[word2id[user_word]-1].argsort()[1:6]+1]
    print(f"Top similar words to '{user_word}': {similar}")
else:
    print(f"'{user_word}' not found in vocabulary.")

Enter a word to find similar words:  deep


Top similar words to 'deep': ['layers', 'learning', 'represent', 'embedding', 'applications']
